In [ ]:
import copy
import os
import random
import time

import cv2 
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image
from sklearn.metrics import confusion_matrix


In [ ]:
df = pd.read_csv("gicsd_labels.csv", skipinitialspace=True)
df.LABEL = df.LABEL.str.strip()
df["IMG_PATH"] = os.path.join(os.getcwd(), "images/") + df.IMAGE_FILENAME 
print(df.head())
full_vis = df[df["LABEL"] == "FULL_VISIBILITY"]
part_vis = df[df["LABEL"] == "PARTIAL_VISIBILITY"]
no_vis = df[df["LABEL"] == "NO_VISIBILITY"]
print(len(full_vis), len(part_vis), len(no_vis))
assert  len(full_vis)+ len(part_vis)+ len(no_vis) == 800

In [ ]:
full_vis_dict = {}
for row in full_vis.iterrows(): 
  path = row[1]["IMG_PATH"]
  name = row[1]["IMAGE_FILENAME"] 
  bgr = cv2.imread(path)
  rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
  gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
  full_vis_dict[name] = (rgb, bgr, gray)
  #print(np.min(gray), np.max(gray))

part_vis_dict = {}
for row in part_vis.iterrows():
  path = row[1]["IMG_PATH"]
  name = row[1]["IMAGE_FILENAME"]
  bgr = cv2.imread(path)
  rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
  gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
  part_vis_dict[name] = (rgb, bgr, gray)
  #print(np.min(gray), np.max(gray))

no_vis_dict = {}
for row in no_vis.iterrows():
  path = row[1]["IMG_PATH"]
  name = row[1]["IMAGE_FILENAME"]
  bgr = cv2.imread(path)
  rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
  gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
  no_vis_dict[name] = (rgb, bgr, gray)
  #print(np.min(gray), np.max(gray))

In [ ]:
key = random.choice(list(full_vis_dict))
rgb, bgr, gray = full_vis_dict[key]

plt.imshow(bgr[:,:,0])
plt.show()

plt.imshow(bgr)
plt.show()

plt.imshow(gray, cmap='gray', vmin=0, vmax=255)
plt.show()

key = random.choice(list(part_vis_dict))
rgb, bgr, gray = part_vis_dict[key]

plt.imshow(bgr[:,:,0])
plt.show()

plt.imshow(bgr)
plt.show()

key = random.choice(list(no_vis_dict))
rgb, bgr, gray = no_vis_dict[key]
plt.imshow(bgr[:,:,0])
plt.show()


plt.imshow(bgr)
plt.show()


# CLAHE (Contrast Limited Adaptive Histogram Equalization)
clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)  # convert from BGR to LAB color space
l, a, b = cv2.split(lab)  # split on 3 different channels
l2 = clahe.apply(l)  # apply CLAHE to the L-channel
lab = cv2.merge((l2,a,b))  # merge channels
img2 = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)  # convert from LAB to BGR
plt.imshow(img2)
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

df_copy = df.copy()

Y = df_copy.pop("LABEL").to_frame()
X = df_copy

# stratified splitting the data to 74% train, 13% val and 13% test for maintaing class imbalance
x_train, x_test, y_train, y_test  = train_test_split(X, Y, random_state=7, stratify=Y, test_size=0.26)
x_test, x_val, y_test, y_val  = train_test_split(x_test, y_test, random_state=7, stratify=y_test, test_size=0.5)

print(len(x_train), len(y_train) ,len(x_test), len(y_test), len(x_val), len(y_val))

print((y_train.LABEL.value_counts()/y_train.LABEL.count())*100)
print((y_test.LABEL.value_counts()/y_test.LABEL.count())*100)
print((y_val.LABEL.value_counts()/y_val.LABEL.count())*100)


In [ ]:
print(y_train.LABEL.value_counts())
weights = [478/478, 478/91, 478/23]
weights

In [ ]:
class CustomImageDataset(Dataset):
    
    def __init__(self, x_df, y_df, transform):
        self.x = x_df.IMG_PATH.to_list()
        self.y = y_df.LABEL.to_list()
        self.transform = transform
        self.class_map = {"FULL_VISIBILITY" : 0,
                          "PARTIAL_VISIBILITY": 1,
                          "NO_VISIBILITY": 2,
                          }
        
        self.clahe = cv2.createCLAHE(clipLimit=2, tileGridSize=(8,8))
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        img = Image.open(self.x[idx]).convert("RGB")        
        if self.transform is not None:
            img = self.transform(img)
            
        class_name = self.y[idx] 
        class_id = self.class_map[class_name]
        class_id = torch.tensor([class_id])
        
        return img, class_id
    

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224), 
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224), 
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

x_df = {'train': x_train, 'val': x_val, 'test': x_test}
y_df = {'train': y_train, 'val': y_val, 'test': y_test}

In [ ]:
dataloaders = { split : DataLoader(
                            CustomImageDataset(x_df[split], y_df[split], data_transforms[split]), 
                            batch_size = 8,
                            num_workers = 4)
                for split in ['train', 'val', 'test']}

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                
                inputs = inputs.to(device)
                labels = labels.to(device)
                labels = labels.reshape(inputs.shape[0])
                #print(labels, labels.shape)
                #print(inputs.shape)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate losss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)
                    #if phase == "val":
                    #    print(preds)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
print(num_ftrs)

# we have 3 output classes
model_ft.fc = nn.Linear(num_ftrs, 3)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss(weight=torch.Tensor(weights).to(device))

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.025 every 10 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.025)

#print(model_ft)

In [ ]:
num_epochs=200
model_ft, hist = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs)

In [ ]:

ohist = [h.cpu().numpy() for h in hist]
plt.title("Validation Accuracy vs. Number of Training Epochs")
plt.xlabel("Training Epochs")
plt.ylabel("Validation Accuracy")
plt.plot(range(1,num_epochs+1), ohist, label="resnet18")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 20))
plt.legend()
plt.show()

In [ ]:

model_ft.eval()   # Set model to evaluate mode

running_loss = 0.0
running_corrects = 0

for inputs, labels in dataloaders["test"]:
    inputs = inputs.to(device)
    labels = labels.to(device)
    labels = labels.reshape(inputs.shape[0])
    #print(labels, labels.shape)
    #print(inputs.shape)
    # zero the parameter gradients
    optimizer_ft.zero_grad()

    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        # Get model outputs and calculate loss
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)

    # statistics
    print(labels.data, preds)
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)
    
    
test_loss = running_loss / len(dataloaders["test"].dataset)
test_acc = running_corrects.double() / len(dataloaders["test"].dataset)

print('{} Loss: {:.4f} Acc: {:.4f}'.format("test", test_loss, test_acc))